In [ ]:
pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib


In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import os

def download_from_drive(service_account_file, folder_id, output_dir):
    creds = service_account.Credentials.from_service_account_file(
        service_account_file,
        scopes=["https://www.googleapis.com/auth/drive"]
    )
    service = build("drive", "v3", credentials=creds)

    query = f"'{folder_id}' in parents and trashed=false"
    results = service.files().list(q=query, fields="files(id, name)").execute()

    os.makedirs(output_dir, exist_ok=True)

    for file in results.get("files", []):
        request = service.files().get_media(fileId=file["id"])
        fh = io.FileIO(os.path.join(output_dir, file["name"]), 'wb')
        downloader = MediaIoBaseDownload(fh, request)

        done = False
        while not done:
            _, done = downloader.next_chunk()
        print(f"Downloaded: {file['name']}")


In [ ]:
if __name__ == "__main__":
    download_from_drive(
        service_account_file = r"",
        folder_id="",
        output_dir="./gdrive_files"
    )


Downloaded: CustomerImportance.csv
Downloaded: transactions.csv
